In [ ]:
import proveit.specialStatementMagic # for %begin_axioms and %end_axioms
from proveit.expression import Operation
from proveit.basiclogic import Forall, In, Equals
from proveit.number import Add, subtract, Multiply, Exponentiate, frac, Abs, Floor
from proveit.number import GreaterThanEquals, LessThanEquals, Mod, ModAbs, Round, IntervalCO, DiscreteContiguousSet
from proveit.number.common import e, i, zero, one, two, pi, two_pi, Integers, NaturalsPos, Reals
from proveit.number import Naturals
from proveit.trigonometry import AngDiff
from proveit.linalg import MatrixProd, ScalarProd, SU, TensorProd, TensorExp
from proveit.statistics import Prob
from proveit.physics.quantum import Bra, Ket, RegisterBra, RegisterKet, Meas, Circuit, Gate, Input, Output, MultiWire
from proveit.physics.quantum.common import Hgate, RegisterSU, QubitRegisterSpace, ket0, \
    CTRL_DN, WIRE_LINK, WIRE_DN, PASS
from proveit.physics.quantum.QFT import InverseFourierTransform
# Note that U_, u_, and t_ are Literals that will eventually be converted to
# Variable counterparts through meta-reasoning.
from proveit.physics.quantum.QPE.common import U_, n_, u_, phase_, t_, Psi_, m_, phase_m_, b_, delta_, two_pow_t,\
    psi_1, psi_k, psi_next, psi_t, alpha_l, U_pow_two_pow_k
from proveit.physics.quantum.QPE import QPE, PhaseEst
from proveit.physics.quantum.QPE.phaseEstOps import Psuccess, Pfail, ModAdd
from proveit.common import U, u, t, eps, a, b, k, l
from IPython.display import display

In [ ]:
%begin_axioms

Let $U$ be a unitary operator that acts on $n$ qubits, $u$ an eigenstate of U with eigenvalue $e^{2 \pi i \varphi}$:

In [ ]:
nInNatPos = In(n_, NaturalsPos) # transferred
unitaryU = In(U_, RegisterSU(n_))  # transferred
uKetRegister = In(Ket(u_), QubitRegisterSpace(n_))  # transferred
phaseInInterval = In(phase_, IntervalCO(zero, one)) # transferred
eigenUu = Equals(MatrixProd(U_, Ket(u_)), ScalarProd(Exponentiate(e, Multiply(two, pi, i, phase_)), Ket(u_))) # transferred
display(nInNatPos)
display(unitaryU)
display(uKetRegister)
display(phaseInInterval)
display(eigenUu)  

Define the Quantum Phase Estimation algorithm, for any $U$, $u$, and $t$ (number of bits of precision), using quantum circuit representations, broken into three stages with recursion in the middle.

In [ ]:
tInNaturalsPos = In(t_, NaturalsPos) # transferred
psiKetsInQRegSpace = Forall(k, In(Ket(psi_k), QubitRegisterSpace(k)), domain=NaturalsPos) # transferred
PsiKetInQRegSpace = In(Ket(Psi_), QubitRegisterSpace(t_)) # transferred
display(tInNaturalsPos)
display(psiKetsInQRegSpace)
display(PsiKetInQRegSpace)

In [ ]:
firstQPE = Circuit([[Input(ket0), Hgate, CTRL_DN, Output(Ket(psi_1))],
                [Input(Ket(u_)), MultiWire(n_), Gate(U_), Output(Ket(u_))]])
firstQPE

In [ ]:
midQPE = Forall(k, 
               Circuit([[Input(Ket(psi_k)), MultiWire(k), PASS, WIRE_DN, PASS, PASS],
                [Input(ket0), Hgate, CTRL_DN, WIRE_LINK, MultiWire(Add(k, one)), Output(Ket(psi_next))],
                [Input(Ket(u_)), MultiWire(n_), Gate(U_pow_two_pow_k), Output(Ket(u_)), PASS, PASS]]),
       domain=NaturalsPos)
midQPE

In [ ]:
finalQPE = Circuit([[Input(Ket(psi_t)), MultiWire(t_), Gate(InverseFourierTransform(t_)), Output(Ket(Psi_))]])
finalQPE

Let $\lvert \Psi \rangle$ be the outcome of the $t$-qubit register of ${\rm QPE}(U, t)$ acting on $\lvert u \rangle$, $m$ be a random variable representing the measurement of Psi with the register interpretted as an integer (via binary representation), and $\varphi_m = 2 \pi m/2^t$ be the random variable phase outcome of ${\rm QPE}(U, t)$:

In [ ]:
mDef = Equals(m_, Meas(Ket(Psi_))) # transferred
phase_m_def = Equals(phase_m_, frac(m_, two_pow_t)) # transferred
display(mDef)
display(phase_m_def)

Let $b$ be the value for $m$ that gives the closest $\varphi_m$ to $\varphi$ without exceeding it, and let $\delta$ be this difference:

In [ ]:
bestDef = Equals(b_, Floor(Multiply(phase_, two_pow_t))) # transferred
deltaDef = Equals(delta_, subtract(phase_, frac(b_, two_pow_t))) # transferred
display(bestDef)
display(deltaDef)

The probability of success is defined as $\theta_m$ being within some epsilon of $\theta$: 

In [ ]:
# transferred
successDef = Forall(eps, Equals(Psuccess(eps), 
                                Prob(LessThanEquals(ModAbs(subtract(m_, b_), two_pow_t), eps), m_)), 
                    domain=NaturalsPos)
# transferred
failDef = Forall(eps, Equals(Pfail(eps), subtract(one, Psuccess(eps))), domain=NaturalsPos)
display(successDef)
display(failDef)             

Let $\alpha_l$ be the amplitude of $\lvert \Psi \rangle$ for the $b \oplus l$ state, where $\oplus$ is defined as addition modulo $2^t$:

In [ ]:
# transferred
modAddDef = Forall((a, b), Equals(ModAdd(a, b), Mod(Add(a, b), two_pow_t)), domain=Integers)
# transferred
alphaDef = Forall(l, Equals(alpha_l, MatrixProd(RegisterBra(ModAdd(b_, l), t_), Ket(Psi_))), domain=Integers)
display(modAddDef)
display(alphaDef)

In [ ]:
%end_axioms